### Orbis and PPP Loan Data Merge

1. Extract matches between the PPP Loan Data and Orbis
2. Get all businesses NOT in Orbis but is in Bay Area AND is Minority-owned
3. Get needed columns from PPP Loan to append to Orbis
4. Extract PPP Loan columns and merge with orbis
5. Add PPP bus not in Orbis with those PPP loan columns
6. Reexport and update everyone

In [1]:
from cleanco import cleanco
import pandas as pd
import re
import os
from os.path import dirname, abspath
from IPython.display import display, HTML # Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

In [48]:
# Get Current Working Directory and Parent Path (for reading files in different folders)
## Source: https://stackoverflow.com/questions/30218802/get-parent-of-current-directory-from-python-script/30218825
d = dirname(dirname(abspath(os.getcwd())))
d

'C:\\Users\\Trang\\Desktop\\afn'

In [3]:
orbis_file = d+'/ppp-loan-data/Orbis database_six cities only.csv'
ppp_file =d+'/ppp-loan-data/out/bay_bus_from_ppp.csv'

In [4]:
orbis_df = pd.read_csv(orbis_file)
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Latitude,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"37° 48' 30.9"" N","122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 18.2"" N","122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 0.2"" N","122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"37° 46' 7.3"" N","122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"37° 48' 51.1"" N","122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland


In [5]:
ppp_df = pd.read_csv(ppp_file)
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NonProfit,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,NaN,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,NaN,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,NaN,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,NaN,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,NaN,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0


In [6]:
# Clean Orbis Name Column
orbis_df['Company Name - Clean'] =orbis_df['Company name Latin alphabet'].apply(lambda x: cleanco(x).clean_name())
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area,Company Name - Clean
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland,CALIFORNIA NURSES ASSOCIATION
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,HONDA CARS OF CORONA
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,SIMI MANAGEMENT
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland,ECONOMY LUMBER CO OF OAKLAND
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,BROADWAY FORD


In [7]:
ppp_df['BorrowerName - Clean'] = ppp_df['BorrowerName'].apply(str).apply(lambda x: cleanco(x).clean_name())
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0,MARKET AUTO TRUCK COLLISION
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0,GOLD RUSH KETTLE KORN
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0,KALIBER LABS
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0,ROSS MCDONALD COMPANY
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0,GATES EISENHART DAWSON


In [8]:
# Find businesses in Orbis databases
ppp_from_orbis = ppp_df[ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_from_orbis.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
19,9412638500,03/12/2021,912.0,PPS,PACIFICA PIZZA,915 1st St,Benicia,CA,94510-3217,0,...,0.00,NaN,94510,No,2021,722513.0,Limited-Service Restaurants,Restaurants and Other Eating Places,7225.0,PACIFICA PIZZA
72,6440528506,03/03/2021,912.0,PPS,435 EL CAMINO REAL INC,435 El Camino Real,South San Francisco,CA,94080-4345,0,...,0.00,NaN,94080,Unanswered,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0,435 EL CAMINO REAL
77,7041187708,05/01/2020,912.0,PPP,"SPORTS PAGE, INC",1431 PLYMOUTH ST,MOUNTAIN VIEW,CA,94043-1227,03/17/2021,...,151052.16,02/12/2021,94043,Unanswered,2020,722410.0,Drinking Places (Alcoholic Beverages),Drinking Places (Alcoholic Beverages),7224.0,SPORTS PAGE
100,8712608401,02/13/2021,912.0,PPS,E & L RESTAURANT INC,1183 S De Anza Blvd Ste 30,San Jose,CA,95129-3659,0,...,0.00,NaN,95129,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0,E & L RESTAURANT
102,5213408402,02/08/2021,912.0,PPS,SIMCO COFFEE LLC,PIER 39 SUITE 213,SAN FRANCISCO,CA,94133,0,...,0.00,NaN,94133,Unanswered,2021,722515.0,Snack and Nonalcoholic Beverage Bars,Restaurants and Other Eating Places,7225.0,SIMCO COFFEE


In [28]:
ppp_columns = ['BorrowerName - Clean','CurrentApprovalAmount', 'YearApproved', 'Race', 'Ethnicity', 'Minority', 'NAICS_4','IndustrySubsector']

### Get Businesses Not in Orbis

In [11]:
ppp_not_in_orbis = ppp_df[~ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_not_in_orbis.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0,MARKET AUTO TRUCK COLLISION
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0,GOLD RUSH KETTLE KORN
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0,KALIBER LABS
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0,ROSS MCDONALD COMPANY
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0,GATES EISENHART DAWSON


In [29]:
ppp_not_in_orbis.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [30]:
ppp_to_orbis_columns = ['BorrowerName','BorrowerAddress','BorrowerCity', 'BorrowerState', 'BorrowerZip','Gender', 'ProjectCity', 'BusinessType']

In [31]:
def extract_columns(df, columns):
    return df[columns]

In [40]:
extracted_ppp = extract_columns(ppp_not_in_orbis, ppp_to_orbis_columns + ppp_columns)
extracted_ppp.head()

,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,Gender,ProjectCity,BusinessType,BorrowerName - Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
0,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,Unanswered,SAN JOSE,Corporation,MARKET AUTO TRUCK COLLISION,15125.0,2020,Unanswered,Unknown/NotStated,Unanswered,8111.0,Automotive Repair and Maintenance
1,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,Male Owned,Benicia,Limited Liability Company(LLC),GOLD RUSH KETTLE KORN,149999.0,2021,White,Not Hispanic or Latino,No,7223.0,Special Food Services
2,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,Male Owned,San Francisco,Corporation,KALIBER LABS,149999.0,2021,Unanswered,Not Hispanic or Latino,No,5415.0,Computer Systems Design and Related Services
3,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,Unanswered,Livermore,Corporation,ROSS MCDONALD COMPANY,149999.0,2021,Unanswered,Unknown/NotStated,Unanswered,5414.0,Specialized Design Services
4,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,Unanswered,San Jose,Partnership,GATES EISENHART DAWSON,149999.0,2021,Unanswered,Unknown/NotStated,Unanswered,5411.0,Legal Services


In [44]:
extracted_ppp_minority = extracted_ppp[extracted_ppp['Minority'] == 'Yes']
extracted_ppp_minority.head()

,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,Gender,ProjectCity,BusinessType,BorrowerName - Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
6,ICHINA,70 Valley Ct,Atherton,CA,94027-6472,Female Owned,Atherton,Corporation,ICHINA,149999.0,2021,Asian,Not Hispanic or Latino,Yes,7225.0,Restaurants and Other Eating Places
7,BANZAI IZAKAYA INC.,1633 Bonanza St,Walnut Creek,CA,94596-4525,Male Owned,Walnut Creek,Corporation,BANZAI IZAKAYA,149999.0,2021,Asian,Not Hispanic or Latino,Yes,7225.0,Restaurants and Other Eating Places
9,RANGOON SUPER STARS LLC,542 Freya Way,Pleasant Hill,CA,94523-1712,Male Owned,Pleasant Hill,Limited Liability Company(LLC),RANGOON SUPER STARS,149999.0,2021,Asian,Not Hispanic or Latino,Yes,7225.0,Restaurants and Other Eating Places
10,VIVA EL ESPANOL SPANISH SCHOOL INC.,3205 Stanley Blvd,Lafayette,CA,94549-3239,Female Owned,Lafayette,Corporation,VIVA EL ESPANOL SPANISH SCHOOL,149999.0,2021,Unanswered,Hispanic or Latino,Yes,6244.0,Child Day Care Services
20,JBR PARTNERS INC,1333 Evans Ave,San Francisco,CA,94124-1705,Male Owned,San Francisco,Corporation,JBR PARTNERS,149999.0,2021,Black or African American,Not Hispanic or Latino,Yes,5418.0,"Advertising, Public Relations, and Related Ser..."


In [45]:
for i in extracted_ppp_minority['BorrowerCity'].str.title().unique():
    print(i)

Atherton
Walnut Creek
Pleasant Hill
Lafayette
San Francisco
Santa Rosa
Fairfield
Hayward
Morgan Hill
San Leandro
Fremont
Portola Valley
El Cerrito
San Rafael
Dublin
Alameda
Pleasanton
Livermore
Burlingame
Oakland
San Ramon
Concord
Sunnyvale
Mountain View
Novato
Daly City
San Mateo
Berkeley
Palo Alto
Campbell
Los Altos
Cotati
San Jose
Vacaville
Saratoga
Stockton
Millbrea
Milpitas
Union City
Richmond
Emeryville
Santa Clara
Los Gatos
Vallejo
Brentwood
South San Francisco
Napa
Hollister
Menlo Park
E Palo Alto
Millbreae
Brisbane
Cupertino
San Bruno
Redwood City
Antioch
Tracy
San Pablo
Yountville
Los Altos Hills
Martinez
Millbrae
Danville
Newark
American Canyon
San Carlos
Benicia
Castro Valley
Petaluma
Newman
Healdsburg
San Lorenzo
Dixon
S San Fran
Sonoma
Alamo
Half Moon Bay
Belmont
Davis
Bodega Bay
Pittsburg
Angwin
Albany
Calistoga
Windsor
Rohnert Park
Hercules
Watsonville
Discovery Bay
San Martin
Sacramento
Foster City
1353 Locust Street
Gilroy
Greenbrae
Mill Valley
Pinole
Patterson
Rio Vi

In [51]:
extracted_ppp_minority.to_csv(d+'/ppp-loan-data/out/ppp-not-in-orbis-minority.csv', index=False)

In [75]:
# Read in New combined files

revised_orbis_ppp_df = pd.read_csv(d+'/ppp-loan-data/out/merged_cities/orbis_ppp_merge.csv')
revised_orbis_ppp_df.head()

C:\Users\Trang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4,5,8,10,11,13,17,18,19,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Company Name - Clean,BorrowerName-Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412.0,LF,2019.0,...,Oakland,CALIFORNIA NURSES ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519.0,LF,2019.0,...,Oakland,HONDA CARS OF CORONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519.0,LF,2019.0,...,Oakland,SIMI MANAGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778.0,LF,2019.0,...,Oakland,ECONOMY LUMBER CO OF OAKLAND,ECONOMY LUMBER CO OF OAKLAND,807700.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4233.0,Lumber and Other Construction Materials Mercha...
4,BROADWAY FORD,No,No,No,No,No,US,4519.0,LF,2008.0,...,Oakland,BROADWAY FORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
revised_orbis_ppp_df.columns

Index(['Company name Latin alphabet', 'Inactive', 'Quoted', 'Branch',
       'OwnData', 'Woco', 'Country ISO code',
       'NACE Rev. 2, core code (4 digits)', 'Consolidation code',
       'Last avail. year',
       'Operating revenue (Turnover)\nth USD Last avail. yr',
       'Number of employees\nLast avail. yr', 'Address line 1\nLatin Alphabet',
       'Address line 2\nLatin Alphabet', 'Postcode\nLatin Alphabet',
       'City\nLatin Alphabet', 'Region in country', 'Latitude', 'Longitude',
       'Website address', 'NAICS 2017, core code (4 digits)',
       'NAICS 2017, core code - description', 'Size classification',
       'Type of entity', 'Ethnic minority owned indicator (in US)',
       'Woman owned indicator (in US)', 'Study Area', 'Company Name - Clean',
       'BorrowerName-Clean', 'CurrentApprovalAmount', 'YearApproved', 'Race',
       'Ethnicity', 'Minority', 'NAICS_4', 'IndustrySubsector'],
      dtype='object')

In [77]:
revised_orbis_ppp_df['Address line 1\nLatin Alphabet'] = revised_orbis_ppp_df['Address line 1\nLatin Alphabet'].str.upper()
revised_orbis_ppp_df['City\nLatin Alphabet'] = revised_orbis_ppp_df['City\nLatin Alphabet'].str.upper()
revised_orbis_ppp_df['Study Area'] = revised_orbis_ppp_df['Study Area'].str.upper()

In [78]:
revised_orbis_ppp_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Company Name - Clean,BorrowerName-Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412.0,LF,2019.0,...,OAKLAND,CALIFORNIA NURSES ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,HONDA CARS OF CORONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,SIMI MANAGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,ECONOMY LUMBER CO OF OAKLAND,ECONOMY LUMBER CO OF OAKLAND,807700.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4233.0,Lumber and Other Construction Materials Mercha...
4,BROADWAY FORD,No,No,No,No,No,US,4519.0,LF,2008.0,...,OAKLAND,BROADWAY FORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
revised_orbis_ppp_df['Study Area'].unique()

array(['OAKLAND', 'REDWOOD CITY', 'RICHMOND', 'SAN FRANCISCO', 'SAN JOSE',
       'SOUTH SAN FRANCISCO', nan], dtype=object)

In [82]:
revised_orbis_ppp_df.to_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-cleaned.csv', index=False)

In [80]:
short_orbis_ppp_df = revised_orbis_ppp_df[revised_orbis_ppp_df['Study Area'].notna()]

In [81]:
short_orbis_ppp_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Company Name - Clean,BorrowerName-Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412.0,LF,2019.0,...,OAKLAND,CALIFORNIA NURSES ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,HONDA CARS OF CORONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,SIMI MANAGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,ECONOMY LUMBER CO OF OAKLAND,ECONOMY LUMBER CO OF OAKLAND,807700.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4233.0,Lumber and Other Construction Materials Mercha...
4,BROADWAY FORD,No,No,No,No,No,US,4519.0,LF,2008.0,...,OAKLAND,BROADWAY FORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
short_orbis_ppp_df.to_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-study-areas.csv', index=False)

Please add in a new column in your spreadsheet called "Operational status"
If the business has an active license based on the business license data from your city list it as 'Active'
If the business has an expired license, list as 'Expired license'
If the business is closed, list as 'Closed'
If the business does not appear in the business license database at all note this as 'Not in license database'